In [ ]:
import numpy as np
import nltk
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.utils import to_categorical

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('movie_reviews')
nltk.download('stopwords')

def load_imdb_dataset():
    positive_reviews = nltk.corpus.movie_reviews.fileids('pos')
    positive_reviews = [nltk.corpus.movie_reviews.raw(fileid) for fileid in positive_reviews]
    negative_reviews = nltk.corpus.movie_reviews.fileids('neg')
    negative_reviews = [nltk.corpus.movie_reviews.raw(fileid) for fileid in negative_reviews]
    dataset = [(review, 'positive') for review in positive_reviews] + [(review, 'negative') for review in negative_reviews]
    return dataset

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return tokens

# Load and preprocess the dataset
dataset = load_imdb_dataset()
random.shuffle(dataset)

reviews, labels = zip(*dataset)
reviews = [" ".join(preprocess_text(review)) for review in reviews]

# Tokenize and pad the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
word_index = tokenizer.word_index
data = pad_sequences(sequences)

# Convert labels to numerical values and one-hot encode them
label_dict = {'positive': 1, 'negative': 0}
labels = [label_dict[label] for label in labels]
labels = to_categorical(labels)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# Define RNN model
embedding_dim = 100
vocab_size = len(word_index) + 1

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=data.shape[1]))
model.add(SimpleRNN(embedding_dim, return_sequences=False))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the RNN model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Test string
test_string = "The movie was one good"
test_string = " ".join(preprocess_text(test_string))
test_sequence = tokenizer.texts_to_sequences([test_string])
test_data = pad_sequences(test_sequence, maxlen=data.shape[1])

# Predict the sentiment
prediction = model.predict(test_data)
predicted_label = "positive" if np.argmax(prediction) == 1 else "negative"
print(f"The predicted sentiment of the test string is: {predicted_label}")


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Epoch 1/10
25/25 [==============================] - 38s 1s/step - loss: 0.7028 - accuracy: 0.5113 - val_loss: 0.7120 - val_accuracy: 0.5175
Epoch 2/10
25/25 [==============================] - 33s 1s/step - loss: 0.5754 - accuracy: 0.7231 - val_loss: 0.6981 - val_accuracy: 0.5225
Epoch 3/10
25/25 [==============================] - 32s 1s/step - loss: 0.3735 - accuracy: 0.9663 - val_loss: 0.7530 - val_accuracy: 0.4500
Epoch 4/10
25/25 [==============================] - 33s 1s/step - loss: 0.2380 - accuracy: 0.9506 - val_loss: 0.7035 - val_accuracy: 0.5350
Epoch 5/10
25/25 [==============================] - 32s 1s/step - loss: 0.1970 - accuracy: 0.9887 - val_loss: 1.0108 - val_accuracy: 0.5075
Epoch 6/10
25/25 [==============================] - 39s 2s/step - loss: 0.0477 - accuracy: 0.9981 - val_loss: 0.8904 - val_accuracy: 0.5000
Epoch 7/10
25/25 [==============================] - 31s 1s/step - loss: 0.0214 - accuracy: 0.9981 - val_loss: 0.9382 - val_accuracy: 0.4950
Epoch 8/10
25/25 [==